# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,15.55,52,75,2.57,US,1694653987
1,1,kingston,17.9970,-76.7936,30.53,74,40,3.09,JM,1694653976
2,2,grytviken,-54.2811,-36.5092,-10.21,67,31,2.94,GS,1694653987
3,3,puerto natales,-51.7236,-72.4875,1.53,99,100,0.42,CL,1694653988
4,4,korla,41.7597,86.1469,21.66,37,39,4.77,CN,1694653988


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & 
                                      (city_data_df["Max Temp"] > 21) &
                                      (city_data_df["Wind Speed"] < 4.5) &
                                      (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,bocaiuva,-17.1078,-43.8150,24.84,44,0,1.72,BR,1694653993
48,48,tukrah,32.5341,20.5791,23.36,67,0,3.97,LY,1694653998
60,60,mitsamiouli,-11.3847,43.2844,24.69,73,0,2.65,KM,1694654001
92,92,kill devil hills,36.0307,-75.6760,24.99,89,0,0.00,US,1694654008
180,180,al jawf,29.5000,38.7500,23.31,60,0,4.49,SA,1694654028


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", 
                               "Country", 
                               "Lat", 
                               "Lng", 
                               "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''



# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
27,bocaiuva,BR,-17.1078,-43.8150,44,
48,tukrah,LY,32.5341,20.5791,67,
60,mitsamiouli,KM,-11.3847,43.2844,73,
92,kill devil hills,US,36.0307,-75.6760,89,
180,al jawf,SA,29.5000,38.7500,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20

params = {
    "categories": categories,
    "apiKey":geoapify_key,
    "limit": limit
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bocaiuva - nearest hotel: No hotel found
tukrah - nearest hotel: No hotel found
mitsamiouli - nearest hotel: Hotel le Paradis
kill devil hills - nearest hotel: Mariner Days Inn & Suites
al jawf - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
remire-montjoly - nearest hotel: Complexe Belova
sydney - nearest hotel: The Westin
carmo do paranaiba - nearest hotel: Hotel Panorâmico
suwayr - nearest hotel: No hotel found
sirjan - nearest hotel: خانه معلم سیرجان
waddan - nearest hotel: No hotel found
dubbo - nearest hotel: Blue Gum Motel
sirte - nearest hotel: فندق المدينة-سرت
ajdabiya - nearest hotel: فندق المدينة
carauari - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
hun - nearest hotel: فندق الهروج
vigo - nearest hotel: Hotel  Silken Axis
siwa oasis - nearest hotel: فندق الكيلانى


,City,Country,Lat,Lng,Humidity,Hotel Name
27,bocaiuva,BR,-17.1078,-43.8150,44,No hotel found
48,tukrah,LY,32.5341,20.5791,67,No hotel found
60,mitsamiouli,KM,-11.3847,43.2844,73,Hotel le Paradis
92,kill devil hills,US,36.0307,-75.6760,89,Mariner Days Inn & Suites
180,al jawf,SA,29.5000,38.7500,60,No hotel found
188,karratha,AU,-20.7377,116.8463,38,Karratha International Hotel
196,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
266,sydney,AU,-33.8679,151.2073,47,The Westin
342,carmo do paranaiba,BR,-19.0008,-46.3161,41,Hotel Panorâmico
379,suwayr,SA,30.1171,40.3893,36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure teh map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols=["Hotel Name" ,"Country"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)